# Working with JSON data from a server
In this lesson, we look at how we can use data from the web.
We will use data from Harvard’s [Caselaw Access Project](https://case.law/) ("CAP").
CAP aims to make all published US courts decisions freely available in a standard, machine-readable format.
CAP and the data format is [documented here](https://case.law/api/).

## Fetching data
To fetch data from the web, we must first import a few libraries:

In [1]:
import requests
import json

We need to specify the URL to the data we want to fetch.

In [2]:
URL = "https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&decision_date_min=2009-01-01&page_size=3"

We include some parameters that specifies which cases we want to load:
- `page_size` is the number of items
- `jurisdiction` is Illinois in this example
- `decision_date_min` is the minimum date, we only want decisions later than this date
- `full_case` include the full text of each case

More parameters are listed in the CAP documentation linked above.

Now, let's fetch the data.

In [3]:
data = requests.get(URL).json()

This step both fetches and decodes the json data in one line. We can also do this step-by-step, to see how the process works.
If you don't want to get into the details at this point, you can skip ahead to the section "Using the data". 
The server response also contains metadata, but we want the content:

In [4]:
response = requests.get(URL)
content = response.content

We can look at the first 100 characters from the raw data. We can see the same data if we open the URL in a web browser:
https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&decision_date_min=2009-01-01&page_size=3

In [5]:
print(content[:100])

b'{"count":2025,"next":"https://api.case.law/v1/cases/?cursor=eyJwIjogWzAuMCwgMTIzMTI4NjQwMDAwMCwgNDI4'


To use the data, we must decode them. We must specify the character set, which is often UTF-8. Then we decode the json format into a Python dictionary.

In [6]:
text = content.decode("utf-8")
data = json.loads(text)

## Using the data
The field `count` contains the number of hits in the database. This is usually different from the number of items we requested.
If the `count` is zero, we don't have any results and need to check the query in the URL.

In [7]:
print(data["count"])

2025


That looks good. Let's fetch the list of cases:

In [8]:
cases = data["results"]

Now we can inspect each case. Let's loop over the cases and get some of the information.
The data contains various metadata about each case, such as the case name and the abbreviated case name.

It's often useful to look at the data in a web browser to get an overview.

In [9]:
for case in cases:
    print("Case name:", case["name_abbreviation"])

Case name: People v. Johnson
Case name: People v. Angarola
Case name: Village of Bensenville v. City of Chicago


## Exercise: docket number

Complete the code below to print the docket number and decision date of each case.
You will need to browse the [data](https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&decision_date_min=2009-01-01&page_size=1").

In [10]:
URL = "https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&decision_date_min=2009-01-01&page_size=3"
data = requests.get(URL).json()

cases = data["results"]
for case in cases:
    print("Case name:", case["name_abbreviation"])
    #your code here

Case name: People v. Johnson
Case name: People v. Angarola
Case name: Village of Bensenville v. City of Chicago


## Working with lists

Each case in the data set contains a list of one or more opinions. These lists are located quite deep in the data structure, in ` case["casebody"]["data"]["opinions"]`. These levels are somewhat like directories in a file tree, and can be seen when browsing the [web interface](https://api.case.law/v1/cases/?jurisdiction=ill&full_case=true&decision_date_min=2009-01-01&page_size=1").

We can look at some of the data for each opinion:

In [11]:
for case in cases:
    print("Case name:", case["name_abbreviation"])
    print("Court:", case["court"]["name"])
    opinions = case["casebody"]["data"]["opinions"]
    for opinion in opinions:
        print(" opinion author:", opinion["author"])

Case name: People v. Johnson
Court: Illinois Appellate Court
 opinion author: JUSTICE LYTTON
 opinion author: None
Case name: People v. Angarola
Court: Illinois Appellate Court
 opinion author: JUSTICE O’MALLEY
Case name: Village of Bensenville v. City of Chicago
Court: Illinois Appellate Court
 opinion author: JUSTICE O’MALLEY
